# Iowa liquor sales prediction for revenue & tax planning

## Data Source
<li> https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy

In [2]:
import numpy as np
from numpy import loadtxt

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
pd.set_option('max_columns', None)

from scipy import stats
from scipy.stats import probplot
from scipy.stats.mstats import zscore

import statsmodels.stats.api as sms
import nltk

import collections as co
from collections import Counter
from collections import OrderedDict

from sklearn import datasets
from sklearn.datasets import make_classification

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.over_sampling import RandomOverSampler

from xgboost import XGBClassifier

#from dask_ml.model_selection import GridSearchCV

/usr/local/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/usr/local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [8]:
df_liquor_sales=pd.read_json('https://data.iowa.gov/resource/spsw-4jax.json')

In [9]:
df_liquor_sales.head()

,:@computed_region_3r5t_5243,:@computed_region_e7ym_nrbf,:@computed_region_i9mz_6gmt,:@computed_region_wnea_7qqw,address,bottle_volume_ml,category,category_name,city,county,county_number,date,im_desc,invoice_line_no,itemno,name,pack,sale_bottles,sale_dollars,sale_gallons,sale_liters,state_bottle_cost,state_bottle_retail,store,store_location,store_location_address,store_location_city,store_location_zip,vendor_name,vendor_no,zipcode
0,832.0,1878.0,345.0,29.0,1101 ARMY POST RD,200,1031080,VODKA 80 PROOF,DES MOINES,Polk,77,2013-07-25,Smirnoff Vodka 80 Prf,S13560300005,37993,Southside Food Mart,48,48,177.12,2.536052,9.6,2.46,3.69,4349,"{'type': 'Point', 'coordinates': [-93.628356, ...",1101 ARMY POST RD,DES MOINES,50315,Diageo Americas,260,50315
1,850.0,1891.0,213.0,35.0,4267 SERGEANT RD,750,1031080,VODKA 80 PROOF,SIOUX CITY,Woodbury,97,2012-10-03,Five O'clock Vodka,S08102400013,35916,Fareway Stores #829 / Sioux City,12,24,119.04,4.755097,18.0,3.31,4.96,4202,"{'type': 'Point', 'coordinates': [-96.356879, ...",4267 SERGEANT RD,SIOUX CITY,51106,Laird And Company,297,51106
2,NaN,NaN,NaN,NaN,114 CENTRAL ST N Suite 1,1750,1031200,VODKA FLAVORED,MARSHALLTOWN,Marshall,64,2014-10-28,New Amsterdam Peach,S22038200026,40313,LIQUOR AND GROCERY DEPOT,6,6,119.88,2.773807,10.5,13.32,19.98,4426,NaN,114 CENTRAL ST N Suite 1,MARSHALLTOWN,50158,E AND J GALLO WINERY,205,50158
3,15.0,178.0,323.0,27.0,1250 N LAKE ST,1750,1041100,AMERICAN DRY GINS,STORM LAKE,Buena Vista,11,2012-03-21,Gordon's Gin London Dry - Pet,S04667100039,30318,Hy-Vee Wine and Spirits / Storm Lake,6,6,94.44,2.773807,10.5,10.49,15.74,2576,"{'type': 'Point', 'coordinates': [-95.200758, ...",1250 N LAKE ST,STORM LAKE,50588,Diageo Americas,260,50588
4,136.0,293.0,421.0,37.0,361 3RD ST NW,1000,1081200,CREAM LIQUEURS,HARTLEY,O'Brien,71,2012-12-12,Bailey's Original Irish Cream,S09447200020,68037,Hartley Wine And Spirits,12,2,50.98,0.528344,2.0,17.00,25.49,4193,"{'type': 'Point', 'coordinates': [-95.482214, ...",361 3RD ST NW,HARTLEY,51346,Diageo Americas,260,51346
